# Evaluation of C3S ECMWF using anomalies

In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import datetime as dt
import dask.array as da
import dask
from dask.diagnostics import ProgressBar
import glob
import math
from collections import Counter

## Anomalies GSOD obs

In [107]:
def generate_gsod_anomalies(station: str = 'CAPT JOSE A  QUINONES GONZALES INTL'):
    # Get the reference
    ref = pd.read_csv(f'/home/jovyan/data/share/heloise/ESA-WP11/GSOD/tp_month_mean_1991_2016_{station}.csv')

    # Get the obs
    obs = pd.read_csv(f'/home/jovyan/data/share/heloise/ESA-WP11/GSOD/tp_month_forecast_{station}.csv')

    anomalies = []
    
    for index, row in obs.iterrows():
        obs_time = dt.datetime.strptime(row['time'], '%Y-%m-%d').date()
        obs_tp = row['tp']

        ref_i = ref.tp[obs_time.month - 1]

        anomaly_abs = obs_tp - ref_i
        anomaly_rel = ((obs_tp - ref_i) / ref_i) * 100

        anomalies.append({'year': obs_time.year, 'month': obs_time.month, 'anomaly_abs':anomaly_abs, 'anomaly_rel': anomaly_rel })

    df_anomalies = pd.DataFrame(anomalies)
    df_anomalies.to_csv(f'/home/jovyan/data/share/heloise/ESA-WP11/Anomaly/anomalies_gsod/anomalies_gsod_{station}.csv')
    
    return df_anomalies


In [108]:
stations = pd.read_csv(f'/home/jovyan/data/share/heloise/ESA-WP11/Anomaly/anomalies_gsod/filtered_stations_cajamarca.csv')

for station in stations.name:
    generate_gsod_anomalies(station)

## Anomalies C3S for the stations points

In [3]:
def get_c3s_run_time(month:int = 1, downscaled:bool = True, hindcast:bool = False, forecast:bool = False) -> xr.Dataset:
    
    """
    Retrieve C3S ECMWF SEAS5 total precipitation data for a specific month.

    Parameters:
        month (int): Month value (1-12) for which to retrieve the data. Defaults to 1 (January).
        downscaled (bool): If True, retrieve the downscaled data. If False, retrieve the original forecast data.
                           Defaults to True.
        hindcast (bool): If True, filter the data to include only hindcast data within the range of 1993-01-01 to
                         2016-12-31. Applicable only when downscaled is True. Defaults to False.
        forecast (bool): If True, filter the data to include only forecast data starting from 2017-01-01.
                         Applicable only when downscaled is True. Defaults to False.

    Returns:
        xr.Dataset: Dataset containing the C3S ECMWF SEAS5 total precipitation data for the specified month.

    """
    
    start_date = np.datetime64(dt.datetime(1993, 1, 1))
    end_date = np.datetime64(dt.datetime(2016, 12, 31))
    forecast_start = np.datetime64(dt.datetime(2017, 1, 1))
    
    if downscaled:
        ds = xr.open_mfdataset(f'/home/jovyan/data/share/Martin/ESA/WP11/c3s_seasonal/downscaled/tp/peru_north/1km/method_002/*/{month:02d}/*.nc', preprocess=preprocess)
        if hindcast:
            ds = ds.sel(run_time=(ds.run_time >= start_date) & (ds.run_time <= end_date))
        if forecast:
            ds = ds.sel(run_time=(ds.run_time >= forecast_start))
    else:
        ds = xr.open_mfdataset( f'/home/jovyan/data/forecast/c3s_seasonal/peru_north/*/*/{month:02d}/*.nc', preprocess=preprocess)

    with ProgressBar():
        ds = ds.load()
        
    return ds


In [4]:
def preprocess(ds,downscaled:bool = True) -> xr.Dataset:
    
    """
    Preprocess the C3S ECMWF SEAS5 total precipitation dataset.

    Parameters:
        ds (xr.Dataset): Dataset containing the C3S ECMWF SEAS5 total precipitation data.
        downscaled (bool): If True, the dataset represents downscaled data. If False, it represents original forecast data.
                           Defaults to True.

    Returns:
        xr.Dataset: Preprocessed dataset.

    """
    
    if 'tprate' in list(ds.keys()):

        nb_days = ds.time.dt.days_in_month
        # m/s to mm/month
        conversion_factor = 1e3 * 3600 * 24

        # Apply conversion
        converted_ds = ds * conversion_factor * nb_days
        converted_ds.attrs["units"] = "mm"
        # rename latitude and longitude to lat /lon (WF format)
        converted_ds = standardize_coordinates(converted_ds)

        # rename variable to tp
        ds = converted_ds.rename_vars(name_dict={'tprate': 'tp'})        
    
    with dask.config.set(**{'array.slicing.split_large_chunks': True}):
        run_time = ds.indexes["time"][0]
        ds = ds.assign_coords({"run_time": run_time})
        ds = ds.expand_dims("run_time")
        ds = ds.assign_coords({"time": [i for i in range(1, 7)]})
        #ds = ds.assign_coords({"offset": ("time", [f"{i}MS" for i in range(6)])})
        ds = ds.rename({"time": "lead_time"})
        # put negative values to zeros
        ds['tp'] = xr.where(ds['tp'] < 0, 0, ds['tp'])

    return ds

In [5]:
def generate_c3s_anomalies():
    
    stations = pd.read_csv(f'/home/jovyan/data/share/heloise/ESA-WP11/Anomaly/anomalies_gsod/filtered_stations_cajamarca.csv')

    anomalies = []
    
    for month in range(1,13):
        
        # Get the C3S ECMWF SEAS5 forecast dataset for the specified month
        ds_forecast = get_c3s_run_time(month, downscaled=True, forecast=True)
        
        # Get the reference dataset for the corresponding month
        ref = xr.open_dataset(f'/home/jovyan/data/share/heloise/ESA-WP11/Anomaly/c3s_ref/tp_c3s_hindcast_month_{month}.nc')

        for year in range(2017,2023):
            
            # Select the forecast data for the specified year and month
            runtime = np.datetime64(dt.datetime(year, month, 1))
            ds_forecast_t = ds_forecast.sel(run_time=(ds_forecast.run_time == runtime))
            
            # Select the corresponding points of the obs stations
            for index, row in stations.iterrows():
                name = row['name']
                lat_obs, lon_obs = row['lat'], row['lon']
            
                ds_station = ds_forecast_t.sel(lon=lon_obs, lat=lat_obs, method='nearest')
                ref_station = ref.sel(lon=lon_obs, lat=lat_obs, method='nearest')
            
                anomaly_abs = ds_station.tp.values - ref_station.tp.values
                anomaly_rel = ((ds_station.tp.values - ref_station.tp.values)/ref_station.tp.values)*100

                anomalies.append({'name': name, 'year': year, 'month': month, 'anomaly_abs':anomaly_abs, 'anomaly_rel': anomaly_rel })
            
    sorted_anomalies = sorted(anomalies, key=lambda x: (x['name'],x['year']))
    df_anomalies = pd.DataFrame(anomalies)
    df_anomalies.to_csv(f'/home/jovyan/data/share/heloise/ESA-WP11/Anomaly/anomalies_ecmwf/all_anomalies.csv')

    return df_anomalies


In [7]:
df_anomalies = generate_c3s_anomalies()

[########################################] | 100% Completed | 1.82 sms
[########################################] | 100% Completed | 2.12 sms
[########################################] | 100% Completed | 1.92 sms
[########################################] | 100% Completed | 2.33 sms
[########################################] | 100% Completed | 2.13 sms
[########################################] | 100% Completed | 1.92 sms
[########################################] | 100% Completed | 1.93 sms
[########################################] | 100% Completed | 2.02 sms
[########################################] | 100% Completed | 1.92 sms
[########################################] | 100% Completed | 2.02 sms
[########################################] | 100% Completed | 2.12 sms
[########################################] | 100% Completed | 2.02 sms


In [8]:
df_anomalies

,name,year,month,anomaly_abs,anomaly_rel
0,MOISES BENZAQUEN RENGIFO,2017,1,"[[57.42100881040096, -6.12664794921875, -1.654...","[[24.140070261037266, -2.1418365246937245, -0...."
1,JUAN SIMONS VELA,2017,1,"[[53.0120267868042, -12.747252146403014, 0.751...","[[19.17131100547796, -4.32911751749577, 0.2206..."
2,CHACHAPOYAS,2017,1,"[[47.23285961151123, -14.45701789855957, 6.939...","[[19.675738023173793, -5.739201931625195, 2.37..."
3,CAPT JOSE A QUINONES GONZALES INTL,2017,1,"[[7.244707683722179, 2.514247417449951, 15.032...","[[26.174754520994686, 8.198650262915349, 51.83..."
4,CADETE GUILLERMO DEL CASTILLO PAREDES,2017,1,"[[54.59946181376776, 1.5704342524210517, 1.807...","[[29.59151883707661, 0.6997514054497231, 0.694..."
...,...,...,...,...,...
571,CAPT JOSE A QUINONES GONZALES INTL,2022,12,"[[-3.873603363831837, 16.042453825473785, 22.6...","[[-15.144435523557354, 61.50666938948254, 73.3..."
572,CADETE GUILLERMO DEL CASTILLO PAREDES,2022,12,"[[-42.49813270568848, -4.926743507385254, -7.2...","[[-21.918830291679146, -2.593342788089036, -3...."
573,GEN FAP ARMANDO REVOREDO IGLESIAS,2022,12,"[[-55.90873138109842, -8.365606625874847, -2.4...","[[-28.244882795397185, -4.097667910807266, -1...."
574,JUANJUI,2022,12,"[[-92.33292897542316, -52.870171864827455, -46...","[[-31.196501125029457, -18.48274886999141, -14..."


## Compare

In [6]:
import re

def inlist(string: str) -> list:
    """
    Convert a string of separated numbers to a list of floats.

    Parameters:
        string (str): A string of separated numbers.

    Returns:
        list: A list of floats.

    """
    # Extract the numbers from the string using regular expression
    numbers = re.findall(r'[-+]?\d*\.?\d+', string)
    # Convert the numbers to floats
    new_list = [float(num) for num in numbers]

    return new_list


In [43]:
def generate_anomalies_comparison():
    """
    Generate a comparison of relative anomalies between C3S and GSOD datasets.
    """
    # Read the C3S anomalies data
    anomalies_c3s = pd.read_csv(f'/home/jovyan/data/share/heloise/ESA-WP11/Anomaly/anomalies_ecmwf/all_anomalies.csv')

    # Read the station names
    stations = pd.read_csv(f'/home/jovyan/data/share/heloise/ESA-WP11/Anomaly/anomalies_gsod/filtered_stations_cajamarca.csv')

    anomalies = []

    for station in stations['name']:
        # Read the GSOD anomalies data for the station
        anomalies_gsod = pd.read_csv(f'/home/jovyan/data/share/heloise/ESA-WP11/Anomaly/anomalies_gsod/anomalies_gsod_{station}.csv')

        # Filter the C3S anomalies for the station
        anomalies_c3s_station = anomalies_c3s.loc[anomalies_c3s['name'] == station]
        
        for year in range(2018, 2023):
            for month in range(1, 13):
                # Get the GSOD anomalies for the specific year and month
                anomalies_gsod_time = anomalies_gsod.loc[(anomalies_gsod['year'] == year) & (anomalies_gsod['month'] == month)]
                if (len(anomalies_gsod_time['anomaly_rel'])!=0):
                    an_gsod_t = anomalies_gsod_time['anomaly_rel'].values[0]
                else:
                    an_gsod_t = np.nan
                
                leads = []                
                min_diff = float('inf')
                min_index = None
                
                for k in range(1, 6):
                    
                    if month > k:
                        runtime = month - k
                        year_k = year
                    else:
                        runtime = 12 + (month - k)
                        year_k = year - 1

                    lead = k

                    # Get the C3S anomalies for the corresponding runtime and lead time
                    anomalies_c3s_station_time = anomalies_c3s_station.loc[(anomalies_c3s_station['year'] == year_k) & (anomalies_c3s_station['month'] == runtime)]

                    an_c3s_t = anomalies_c3s_station_time['anomaly_rel'].values[0]
                    an_c3s_t_list = inlist(an_c3s_t)

                    leads.append(an_c3s_t_list[k]) # k is the lead time
                    
                    # best lead
                    diff = abs(an_gsod_t - float(leads[k-1]))
                    if diff < min_diff:
                        min_diff = diff
                        min_index = k
                        
                # leads okay
                leads_good = True
                good_lead_index = None

                #threshold = 0.5 * an_gsod_t

                # Find the index of the last lead time where the condition is met
                for k, lead_value in enumerate(leads):
                    #if (an_gsod_t > 0 and float(lead_value) < threshold) or (an_gsod_t < 0 and float(lead_value) > threshold):
                    if (math.isnan(an_gsod_t)) or (an_gsod_t > 0 and float(lead_value) < 0) or (an_gsod_t < 0 and float(lead_value) > 0):
                        leads_good = False
                        break
                    good_lead_index = k + 1

                
                anomalies.append({'name': station, 'year': year, 'month': month, 'anomaly_gsod_rel (%)': an_gsod_t, 'anomaly_c3s_rel_leads_1_5 (%)': leads, 'good_sign_lead': good_lead_index, 'best_lead': min_index, 'diff (%)': min_diff})

    df_anomalies = pd.DataFrame(anomalies)
    df_anomalies.to_csv(f'/home/jovyan/data/share/heloise/ESA-WP11/Anomaly/anomalies_compared.csv')


In [41]:
generate_anomalies_comparison()

### Analyse leads results

How much time it detects more rain ? a lack of rain ?

If it's raining and it detects if, until which lead it see it ? idem dry month ?

idem by station 

- annual
- seasonnal
- El Nino / no El Nino

In [83]:
# Read the C3S anomalies data
anomalies_compared = pd.read_csv(f'/home/jovyan/data/share/heloise/ESA-WP11/Anomaly/anomalies_compared.csv')

rain, rain_detected = 0, 0
no_rain , no_rain_detected = 0, 0
rain_good_lead, no_rain_good_lead = [], []

for index, q in enumerate(anomalies_compared['anomaly_gsod_rel (%)']):
    if (q > 0) :
        rain = rain + 1
        if (not math.isnan(anomalies_compared.good_sign_lead[index])):
            rain_detected = rain_detected + 1
            rain_good_lead.append(anomalies_compared.good_sign_lead[index])
    elif (q < 0) :
        no_rain = no_rain + 1
        if (not math.isnan(anomalies_compared.good_sign_lead[index])):
            no_rain_detected = no_rain_detected + 1
            no_rain_good_lead.append(anomalies_compared.good_sign_lead[index])

# Calculate the most common lead time for both positive and negative anomalies
rain_count = Counter(rain_good_lead)
rain_maj_lead = rain_count.most_common(1)[0][0]
no_rain_count = Counter(no_rain_good_lead)
no_rain_maj_lead = no_rain_count.most_common(1)[0][0]

# Print the results
print(f'Positive precipitation detected {rain_detected} times over {rain}, ie {rain_detected/rain * 100:.2f}%.')
print(f'Most of the time, it is detected up to month + {rain_maj_lead}.\n')
print(f'Negative precipitation detected {no_rain_detected} times over {no_rain}, ie {no_rain_detected/no_rain * 100:.2f}%.')
print(f'Most of the time, it is detected up to month + {no_rain_maj_lead}.')


Positive precipitation detected 85 times over 173, ie 49.13%.
Most of the time, it is detected up to month + 5.0.

Negative precipitation detected 151 times over 248, ie 60.89%.
Most of the time, it is detected up to month + 5.0.


In [ ]:
# Read the C3S anomalies data
anomalies_compared = pd.read_csv('/home/jovyan/data/share/heloise/ESA-WP11/Anomaly/anomalies_compared.csv')

# Function to calculate statistics for each station
def calculate_station_statistics(data):
    rain, rain_detected = 0, 0
    rain_good_lead = []
    no_rain , no_rain_detected = 0, 0
    no_rain_good_lead = []

    for index, q in data['anomaly_gsod_rel (%)'].iteritems():
        if q > 0:
            rain = rain + 1
            if not pd.isnull(data['good_sign_lead'][index]):
                rain_detected = rain_detected + 1
                rain_good_lead.append(data['good_sign_lead'][index])
        elif q < 0:
            no_rain = no_rain + 1
            if not pd.isnull(data['good_sign_lead'][index]):
                no_rain_detected = no_rain_detected + 1
                no_rain_good_lead.append(data['good_sign_lead'][index])

    rain_count = Counter(rain_good_lead)
    rain_maj_lead = rain_count.most_common(1)[0][0]
    no_rain_count = Counter(no_rain_good_lead)
    no_rain_maj_lead = no_rain_count.most_common(1)[0][0]

    return {'Positive precipitation detected': rain_detected,
            'Total positive precipitation': rain,
            'Positive precipitation percentage': rain_detected / rain * 100,
            'Most of the time, detected up to month +': rain_maj_lead,
            'Negative precipitation detected': no_rain_detected,
            'Total negative precipitation': no_rain,
            'Negative precipitation percentage': no_rain_detected / no_rain * 100,
            'Most of the time, detected up to month +': no_rain_maj_lead}

# Group the data by station name and calculate statistics for each group
statistics_by_station = anomalies_compared.groupby('name').apply(calculate_station_statistics)

# Print the results for each station
for station, statistics in statistics_by_station.items():
    print(f'Station: {station}')
    for key, value in statistics.items():
        print(f'{key}: {value}')
    print('\n')
